In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
def createFolder(directory):
    #저장 폴더 생성
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory' + directory)

In [3]:
def changePath(directory):
    #데이터 폴더 선택
    current_path = os.getcwd()
    change_path = current_path+directory
    try:
        if os.path.exists(directory[1:]):
            os.chdir(change_path)
    except OSError:
        print('Error: Changing Path ' + directory)

In [4]:
def cell_separate(df, cell_cnt):
    cell_index = df.index[df['CellNo']==(cell_cnt+1)].tolist()
    df_cell = df.iloc[cell_index, :]
    newf_cell = df_cell[column_list]
    
    return newf_cell

In [5]:
# Initialization for system model.
A = 1
H = 1
Q = 0.1
R = 7
# Initialization for estimation.
x_0 = 10  # 14 for book.
P_0 = 6

def _filter(z_meas, x_esti, P):
    """Kalman Filter Algorithm for One Variable."""
    # (1) Prediction.
    x_pred = A * x_esti
    P_pred = A * P * A + Q

    # (2) Kalman Gain.
    K = P_pred * H / (H * P_pred * H + R)

    # (3) Estimation.
    x_esti = x_pred + K * (z_meas - H * x_pred)

    # (4) Error Covariance.
    P = P_pred - K * H * P_pred

    return x_esti, P

def kalman_filter(df, noise_column, _filter):
    df_len = len(df)
    x_0 = df.loc[0, noise_column]
    r_meas_save = np.zeros(df_len)
    r_esti_save = np.zeros(df_len)
    
    x_esti, P = None, None
    for j in range(df_len):
        r_meas = df.loc[j, noise_column]
        if j == 0 :
            x_esti, P = x_0, P_0
        else:
            x_esti, P = _filter(r_meas, x_esti, P)
        r_meas_save[j] = r_meas
        r_esti_save[j] = x_esti
        
    df.loc[:, noise_column] = r_esti_save
    
    return df

In [6]:
def UpperLowerLimit(df):
    df_len = len(df)
    
    for i in range(df_len):
        if not i==(df_len-1):
            if not df.loc[i+1, noise_column]==None:
                error = ((df.loc[i+1, noise_column]-df.loc[i, noise_column])/df.loc[i, noise_column])
                
                if i==0:
                    fix_rate = 0
                if i>=1:
                    fix_rate = abs((df.loc[i, noise_column]-df.loc[i-1, noise_column])/df.loc[i-1, noise_column])

            if abs(error)>=0.2:
                if error < 0:
                    df.loc[i+1, noise_column] = df.loc[i, noise_column]*(1-fix_rate)
                if error > 0:
                    df.loc[i+1, noise_column] = df.loc[i, noise_column]*(1+fix_rate)
                    
    return df

In [7]:
if __name__ =='__main__':
    print('생성 폴더 이름 : ')
    folder_name = input()
    createFolder(folder_name)
    current_path = os.getcwd()
    print(current_path)
    print('셀분리 데이터 폴더 입력(\ 포함)')
    data_folder_path = input()
    changePath(data_folder_path) #상하제한 수정할 데이터 폴더로 이동
    print(os.getcwd())

생성 폴더 이름 : 
test2
F:\Personal\BatteryDatasetImplementation-master\omona\work010
셀분리 데이터 폴더 입력(\ 포함)
\testdata
F:\Personal\BatteryDatasetImplementation-master\omona\work010\testdata


In [8]:
    print('셀 교체 diff offset(20):')
    diff_offset = int(input())
    print('셀 교체 y_offset(25):')
    y_offset = int(input())
    print('셀 교체 x_offset(15):')
    x_offset = int(input())
    print('셀 교체 total_error_offset(12):')
    total_error_offset = int(input())
    print('데이터 파일 선택 경로 전체:1 선택:0 :')
    data_select = input()
    print('칼만필터 적용 여부 yes:1 no:0 :')
    KF = int(input())
    print('상하제한 적용 여부 yes:1 no:0 :')
    ULL = int(input())

셀 교체 diff offset(20):
20
셀 교체 y_offset(25):
20
셀 교체 x_offset(15):
15
셀 교체 total_error_offset(12):
12
데이터 파일 선택 경로 전체:1 선택:0 :
1
칼만필터 적용 여부 yes:1 no:0 :
1
상하제한 적용 여부 yes:1 no:0 :
0


In [9]:
    if data_select == '1':
        file_list = os.listdir(os.getcwd())
        print(file_list)
    if data_select == '0':
        print(*os.listdir(os.getcwd()), sep='\n')
        print('SELECT DATA FILE : ')
        file_list = []
        select_file = input()
        file_list.append(select_file)
        print(file_list)

['m2_s2_bank_01c70366fc4e1a439e0b4516698baadf__site_40ab7c891c559b478e96f8bb5c682ca0.csv']


In [10]:
    file_count = len(file_list)
    print(file_count)
    save_path = current_path + '\\' + folder_name
    print(save_path)
    current_path = os.getcwd()
    print(current_path)

1
F:\Personal\BatteryDatasetImplementation-master\omona\work010\test2
F:\Personal\BatteryDatasetImplementation-master\omona\work010\testdata


In [11]:
    noise_column = 'ResistValue'
    column_list = ['KeyTime', 'ResistValue', 'VoltValue', 'TempValue']

In [12]:
    for i in range(file_count):
        
        file_name = file_list[i]
        delete_csv_name = file_name[:-4]
        print(delete_csv_name)
        
        df = pd.read_csv(file_name, encoding = 'utf8')
        
        max_cell = max(df['CellNo'])
        df_full_len = len(df)
        
        #df = df.interpolate(method='linear', limit_direction='forward')
        
        for cell_cnt in range(max_cell):
            
            ch_times = []
            change_num = 0
            
            #1. 셀 분리
            newf_cell = cell_separate(df, cell_cnt)
            
            #2. 결측치 처리
            newf_cell = newf_cell.interpolate(method='linear', limit_direction='forward')
            
            newf_cell = newf_cell.reset_index(drop=True)
            
            df_len = len(newf_cell)
            
            #3. 셀 교체 확인
            for df_len_cnt in range(df_len):
                
                if (df_len_cnt >= (x_offset + 1)) & (df_len_cnt <= (df_len-x_offset)) :
                    
                    total_error_bef = 0
                    total_error_next = 0
                    total_error = 0
                    test_error = abs((newf_cell.loc[df_len_cnt+1, noise_column]-newf_cell.loc[df_len_cnt, noise_column])/newf_cell.loc[df_len_cnt, noise_column]*100)
                    
                    if test_error >= y_offset:
                        for h in range(x_offset): 
                            error_bef = abs((newf_cell.loc[df_len_cnt+(h+1), noise_column]-newf_cell.loc[df_len_cnt, noise_column])/newf_cell.loc[df_len_cnt, noise_column]*100)
                            error_next = abs((newf_cell.loc[df_len_cnt-(h+1), noise_column]-newf_cell.loc[df_len_cnt, noise_column])/newf_cell.loc[df_len_cnt, noise_column]*100)
                            
                            total_error_bef += error_bef
                            total_error_next += error_next
                            
                            if error_bef>=y_offset:
                                total_error += 1
                            if error_next>=y_offset:
                                total_error += 1
                            if h==9:
                                change_check_rate = abs((total_error_bef-total_error_next)/total_error_next*100)
                                if change_check_rate >=diff_offset :
                                    change_check = True
                                else:
                                    change_check = False
                                print('df_len_cnt:', df_len_cnt, ' H:', h+1, ' Total error:', total_error,
                                      'Error check: ', change_check_rate, change_check)
                            
                        #위치 확인
                        if (total_error > total_error_offset)& (change_check==True):
                            print('******CHANGE TIMES')
                            if change_num == 0:
                                change_num += 1
                                ch_times.append(df_len_cnt+1)
                                print('ch_times : ', ch_times, 'ch_num : ', change_num,
                                      'Resist : ', newf_cell.loc[df_len_cnt, noise_column])
                            if (change_num >=1) & (ch_times[change_num-1]+10<=df_len_cnt):
                                change_num += 1
                                ch_times.append(df_len_cnt+1)
                                print('ch_times : ', ch_times, 'ch_num : ', change_num,
                                      'Resist : ', newf_cell.loc[df_len_cnt, noise_column])
                                
            ch_times.insert(0, None)
            ch_times.append(None)
            print('CHECK TIMES')
            print(ch_times)
            
            for cell_ch_cnt in range(change_num+1):
                
                KFOX = 'X'
                ULLOX = 'X'
                print('adf')
                print(ch_times[cell_ch_cnt+1])
                
                newf_ch_cell = newf_cell.iloc[ch_times[cell_ch_cnt]:ch_times[cell_ch_cnt + 1], :]
                
                newf_ch_cell = newf_ch_cell.reset_index(drop=True)
                    
                print(newf_ch_cell)

                
                if KF == 1:
                    newf_ch_cell = kalman_filter(newf_ch_cell, noise_column, _filter)
                    KFOX = 'O'
                
                if ULL == 1:
                    newf_ch_cell = UpperLowerLimit(newf_ch_cell)
                    ULLOX = 'X'
                    
                os.chdir(save_path)
                save_file = delete_csv_name + '_' + str(cell_cnt+1).zfill(2) + '_replace_' + str(cell_ch_cnt+1).zfill(2) + '_KF_' + KFOX + '_ULL_' + ULLOX + '.csv'
                newf_ch_cell.to_csv(save_file, index=False, encoding='utf8')
                os.chdir(current_path)

m2_s2_bank_01c70366fc4e1a439e0b4516698baadf__site_40ab7c891c559b478e96f8bb5c682ca0
df_len_cnt: 184  H: 10  Total error: 10 Error check:  186.9757174392941 True
******CHANGE TIMES
ch_times :  [185] ch_num :  1 Resist :  0.545
CHECK TIMES
[None, 185, None]
adf
185
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.181       4.47       17.3
1    2020-02-15 14:02        0.180       4.47       17.7
2     2020-02-16 2:01        0.184       4.47       17.2
3    2020-02-16 14:00        0.184       4.47       18.1
4     2020-02-17 2:00        0.175       4.47       18.0
..                ...          ...        ...        ...
180   2020-07-02 4:58        0.586       4.47       17.8
181  2020-07-02 16:57        0.585       4.47       18.1
182   2020-07-03 4:57        0.586       4.47       17.8
183  2020-07-03 13:10        0.601       4.46       18.6
184  2020-07-03 13:17        0.545       4.46       18.6

[185 rows x 4 columns]
adf
None
              KeyTim

df_len_cnt: 182  H: 10  Total error: 10 Error check:  156.93069306930693 True
******CHANGE TIMES
ch_times :  [183] ch_num :  1 Resist :  0.742
CHECK TIMES
[None, 183, None]
adf
183
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.389       4.46      -20.0
1    2020-02-15 14:02        0.386       4.46      -20.0
2     2020-02-16 2:01        0.386       4.47      -20.0
3    2020-02-16 14:00        0.387       4.45      -20.0
4     2020-02-17 2:00        0.389       4.45      -20.0
..                ...          ...        ...        ...
178   2020-07-02 4:58        0.617       4.45      -20.0
179  2020-07-02 16:57        0.616       4.46      -20.0
180   2020-07-03 4:57        0.621       4.46      -20.0
181  2020-07-03 13:10        0.626       4.45      -20.0
182  2020-07-03 13:17        0.742       4.45      -20.0

[183 rows x 4 columns]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0    2020-07-03 13:25        0.424       4.45

CHECK TIMES
[None, None]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.378       4.47       16.8
1    2020-02-15 14:02        0.379       4.46       17.0
2     2020-02-16 2:01        0.379       4.47       16.7
3    2020-02-16 14:00        0.379       4.46       17.4
4     2020-02-17 2:00        0.383       4.47       17.5
..                ...          ...        ...        ...
541   2021-02-01 8:50        0.446       4.47       21.8
542  2021-02-01 20:49        0.446       4.47       21.8
543   2021-02-02 8:49        0.447       4.47       21.9
544  2021-02-02 20:48        0.446       4.47       21.8
545   2021-02-03 8:48        0.447       4.47       21.9

[546 rows x 4 columns]
df_len_cnt: 181  H: 10  Total error: 10 Error check:  1063.2075471698233 True
******CHANGE TIMES
ch_times :  [182] ch_num :  1 Resist :  0.5760000000000001
CHECK TIMES
[None, 182, None]
adf
182
              KeyTime  ResistValue  VoltValue  TempValue
0     2

df_len_cnt: 181  H: 10  Total error: 10 Error check:  3058.6206896551694 True
******CHANGE TIMES
ch_times :  [182] ch_num :  1 Resist :  0.607
df_len_cnt: 182  H: 10  Total error: 20 Error check:  35.31264057579844 True
******CHANGE TIMES
CHECK TIMES
[None, 182, None]
adf
182
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.408       4.46      -20.0
1    2020-02-15 14:02        0.409       4.47      -20.0
2     2020-02-16 2:01        0.411       4.47      -20.0
3    2020-02-16 14:00        0.411       4.46      -20.0
4     2020-02-17 2:00        0.408       4.46      -20.0
..                ...          ...        ...        ...
177  2020-07-01 16:58        0.613       4.47      -20.0
178   2020-07-02 4:58        0.612       4.47      -20.0
179  2020-07-02 16:57        0.614       4.47      -20.0
180   2020-07-03 4:57        0.615       4.47      -20.0
181  2020-07-03 13:10        0.607       4.47      -20.0

[182 rows x 4 columns]
adf
None
      

df_len_cnt: 180  H: 10  Total error: 10 Error check:  3627.499999999997 True
******CHANGE TIMES
ch_times :  [181] ch_num :  1 Resist :  0.614
df_len_cnt: 181  H: 10  Total error: 12 Error check:  96.62698412698407 True
******CHANGE TIMES
CHECK TIMES
[None, 181, None]
adf
181
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.384       4.49      -20.0
1    2020-02-15 14:02        0.385       4.50      -20.0
2     2020-02-16 2:01        0.388       4.49      -20.0
3    2020-02-16 14:00        0.391       4.49      -20.0
4     2020-02-17 2:00        0.388       4.50      -20.0
..                ...          ...        ...        ...
176  2020-07-01 16:58        0.614       4.49      -20.0
177   2020-07-02 4:58        0.616       4.51      -20.0
178  2020-07-02 16:57        0.613       4.50      -20.0
179   2020-07-03 4:57        0.611       4.50      -20.0
180  2020-07-03 13:10        0.614       4.50      -20.0

[181 rows x 4 columns]
adf
None
       

...

CHECK TIMES
[None, None]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.361       4.46      -20.0
1    2020-02-15 14:02        0.336       4.46      -20.0
2     2020-02-16 2:01        0.391       4.46      -20.0
3    2020-02-16 14:00        0.391       4.46      -20.0
4     2020-02-17 2:00        0.355       4.47      -20.0
..                ...          ...        ...        ...
541   2021-02-01 8:50        0.459       4.48      -20.0
542  2021-02-01 20:49        0.458       4.46      -20.0
543   2021-02-02 8:49        0.459       4.47      -20.0
544  2021-02-02 20:48        0.459       4.46      -20.0
545   2021-02-03 8:48        0.461       4.47      -20.0

[546 rows x 4 columns]
df_len_cnt: 181  H: 10  Total error: 9 Error check:  1149.0566037735841 True
df_len_cnt: 182  H: 10  Total error: 20 Error check:  18.420375611011057 False
CHECK TIMES
[None, None]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-

df_len_cnt: 181  H: 10  Total error: 10 Error check:  7552.380952380943 True
******CHANGE TIMES
ch_times :  [182] ch_num :  1 Resist :  0.601
df_len_cnt: 182  H: 10  Total error: 20 Error check:  61.84971098265897 True
******CHANGE TIMES
CHECK TIMES
[None, 182, None]
adf
182
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.343       4.49      -20.0
1    2020-02-15 14:02        0.402       4.46      -20.0
2     2020-02-16 2:01        0.374       4.48      -20.0
3    2020-02-16 14:00        0.447       4.47      -20.0
4     2020-02-17 2:00        0.363       4.48      -20.0
..                ...          ...        ...        ...
177  2020-07-01 16:58        0.600       4.47      -20.0
178   2020-07-02 4:58        0.599       4.46      -20.0
179  2020-07-02 16:57        0.599       4.48      -20.0
180   2020-07-03 4:57        0.601       4.48      -20.0
181  2020-07-03 13:10        0.601       4.47      -20.0

[182 rows x 4 columns]
adf
None
       

CHECK TIMES
[None, 181, None]
adf
181
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.400       4.46      -20.0
1    2020-02-15 14:02        0.349       4.50      -20.0
2     2020-02-16 2:01        0.350       4.49      -20.0
3    2020-02-16 14:00        0.364       4.48      -20.0
4     2020-02-17 2:00        0.325       4.51      -20.0
..                ...          ...        ...        ...
176  2020-07-01 16:58        0.611       4.50      -20.0
177   2020-07-02 4:58        0.610       4.49      -20.0
178  2020-07-02 16:57        0.608       4.50      -20.0
179   2020-07-03 4:57        0.610       4.50      -20.0
180  2020-07-03 13:10        0.617       4.49      -20.0

[181 rows x 4 columns]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0    2020-07-03 13:17        0.474       4.50      -20.0
1    2020-07-03 13:25        0.454       4.50      -20.0
2    2020-07-03 13:31        0.462       4.50      -20.0
3     2020-07-04 

adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0    2020-07-03 13:25        0.451       4.47       18.6
1    2020-07-03 13:31        0.460       4.47       18.6
2     2020-07-04 1:31        0.461       4.47       18.1
3    2020-07-04 13:30        0.460       4.47       18.2
4     2020-07-05 1:30        0.457       4.46       18.0
..                ...          ...        ...        ...
358   2021-02-01 8:50        0.449       4.47       22.7
359  2021-02-01 20:49        0.449       4.47       22.5
360   2021-02-02 8:49        0.448       4.46       22.5
361  2021-02-02 20:48        0.448       4.47       22.7
362   2021-02-03 8:48        0.451       4.47       22.6

[363 rows x 4 columns]
df_len_cnt: 179  H: 10  Total error: 10 Error check:  715.5913978494617 True
******CHANGE TIMES
ch_times :  [180] ch_num :  1 Resist :  0.602
CHECK TIMES
[None, 180, None]
adf
180
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.355       4.48 

CHECK TIMES
[None, 179, None]
adf
179
              KeyTime  ResistValue  VoltValue  TempValue
0     2020-02-15 2:02        0.000       4.47      -20.0
1    2020-02-15 14:02        0.355       4.49      -20.0
2     2020-02-16 2:01        0.350       4.49      -20.0
3    2020-02-16 14:00        0.353       4.49      -20.0
4     2020-02-17 2:00        0.354       4.49      -20.0
..                ...          ...        ...        ...
174  2020-07-01 16:58        0.608       4.49      -20.0
175   2020-07-02 4:58        0.608       4.50      -20.0
176  2020-07-02 16:57        0.608       4.48      -20.0
177   2020-07-03 4:57        0.609       4.49      -20.0
178  2020-07-03 13:10        0.593       4.49      -20.0

[179 rows x 4 columns]
adf
None
              KeyTime  ResistValue  VoltValue  TempValue
0    2020-07-03 13:25        0.443       4.47      -20.0
1    2020-07-03 13:31        0.451       4.49      -20.0
2     2020-07-04 1:31        0.448       4.49      -20.0
3    2020-07-04 1